In [1]:
import ira
import inspect
from typing import Union, List
from datetime import timedelta
import requests

%alphalab dark
import alpha.utils.plot_helpers

(c) 2017, AppliedAlpha Infrastructure for Quantitative Research and Analysis, ver. master: 
0.1536

In [2]:
ohlc = pd.read_csv('../tests/data/EURUSD.csv', parse_dates=True, index_col='time')
ohlc1 = pd.read_csv('../tests/data/RM1.csv', parse_dates=True, index_col='time')

In [ ]:
LookingGlass(ohlc).look().hover(n=5)

In [ ]:
LookingGlass(ohlc1).look().hover()

<pre>
2020-08-17 05:58:00     EURUSD      1000    1.185980          0.0                                  Test long order
2020-08-17 05:58:00     EURUSD      -500    1.186230          0.0      part_take long at 1.18623 by LIMIT -> [500]
2020-08-17 06:03:00     EURUSD      -500    1.186480          0.0  take long at 1.1864799999999998 by LIMIT -> [0]
2020-08-17 07:32:00     EURUSD     -1000    1.184655          0.0                                 Test short order
2020-08-17 10:28:00     EURUSD      1000    1.185655          0.0                           stop short at 1.185655
</pre>